
# Homework 5

### Exercise 1

#### (a)

In [11]:
import numpy as np
import tabulate as tab

columns = 20
lines = 20
total_states = columns*lines

pU = np.zeros((total_states,total_states))
pD = np.zeros((total_states,total_states))
pL = np.zeros((total_states,total_states))
pR = np.zeros((total_states,total_states))

greylight_areas = [47,48,49,50,60,61,62,63,64,65,66,67,70,80,90,100,110,120,121,122,123,124,125,126,127,130,147,150,167,170,187,190,207,210,227,230,247,250,267,268,269,270]
greydark_areas = [81,82,83,84,85,86,87,88,89,101,102,103,104,105,106,107,108,109,128,129,148,149,168,169,188,189,208,109,228,229,240,248,249]
goal = [18,19,39]

actions_name = {0: 'U', 1: 'D', 2: 'L', 3: 'R'}
actions_matrix = {'U': pU, 'D': pD, 'L': pL, 'R': pR}

cost = np.full((total_states, len(actions_name)), 0.05)

def calcCost():
    for pos in greylight_areas:
        cost[pos] = 0.5
    
    for pos in greydark_areas:
        cost[pos] = 1
    
    for pos in goal:
        cost[pos] = 0

def convertedPos(x,y):
    return x*columns + y

def getAdjacents(x,y):
    valid_adjacents = dict()
    adjacents = [(x-1,y),(x+1,y),(x,y-1),(x,y+1)]
    
    for index, adjacent in enumerate(adjacents):
        action_name = actions_name[index]
        
        if 0 <= adjacent[0] < columns and 0 <= adjacent[1] < columns:
            valid_adjacents[action_name] = convertedPos(adjacent[0],adjacent[1])   
        else:
            valid_adjacents[action_name] = None

    return valid_adjacents

for x in range(columns):
    for y in range(lines):
        n_state = convertedPos(x,y)
        adjacents = getAdjacents(x,y)
               
        for action in actions_matrix:
            current_matrix = actions_matrix[action]
            current_matrix[n_state,n_state] += 0.02
            
            for adjacent in adjacents:
                adjacent_state = adjacents[adjacent]
            
                if adjacent == action and adjacents[action] != None:
                    current_matrix[n_state,adjacent_state] += 0.92
                    continue
                elif adjacent == action:
                    current_matrix[n_state,n_state] += 0.92
                    continue
                
                if adjacents[adjacent] != None:
                    current_matrix[n_state,adjacent_state] += 0.02
                    continue
                else:
                    current_matrix[n_state,n_state] += 0.02
                    continue

                

In [12]:
alpha = 0.1
gamma = 0.95
total_actions = len(actions_matrix)
Q = np.zeros((total_states, total_actions))
Q[convertedPos(3,16)] = np.array([0.25,0.32,0.32,0.25])
Q[convertedPos(4,16)] = np.array([0.29,0.36,0.36,0.29])

def getMin(state_values):  
    min_value = np.amin(state_values)  
    min_indexes = np.where(state_values == min_value)

    return np.random.choice(min_indexes[0])
             
def minAction(Qfunc, state, epsilon = 0.1):
    next_action = getMin(Qfunc[state])
    
    choice = np.random.choice(['exploit', 'explore'], 1, p = [1 - epsilon, epsilon])
    
    if choice == 'exploit':
        return next_action
    elif choice == 'explore':
        return np.random.randint(len(actions_name.keys()))
          
def qvals_Qlearning(state, next_action, Q, next_state):
    #next_action = minAction(Q, state)
    next_action_name = actions_name[next_action]
    next_action_matrix = actions_matrix[next_action_name]
    #next_state = np.argmax(next_action_matrix[state])
    
    Q[state][next_action] += alpha * (cost[state][next_action] + gamma * np.amin(Q[next_state]) - Q[state][next_action])

    
state = convertedPos(3,16)
next_state = convertedPos(4,16)
qvals_Qlearning(state, 1, Q, next_state)
    
print(Q[state])

[0.25    0.32055 0.32    0.25   ]


#### (b)

In [13]:
Qsarsa= np.zeros((total_states, total_actions))
Qsarsa[convertedPos(3,16)] = np.array([0.25,0.32,0.32,0.25])
Qsarsa[convertedPos(4,16)] = np.array([0.29,0.36,0.36,0.29])

def qvals_sarsa(state, next_action, Q, next_state):
    #next_action = minAction(Q, state)
    next_action_name = actions_name[next_action]
    next_action_matrix = actions_matrix[next_action_name]
    #next_state = np.argmax(next_action_matrix[state])
    
    Q[state][next_action] += alpha * (cost[state][next_action] + gamma * Q[next_state][next_action] - Q[state][next_action])

    
state = convertedPos(3,16)
next_state = convertedPos(4,16)
qvals_sarsa(state, 1, Qsarsa,  next_state)

print(Qsarsa[state])

[0.25   0.3272 0.32   0.25  ]


#### (c)

An off-policy learner learns the value of the optimal policy independently of the agent's actions. Q-learning is an off-policy learner. An on-policy learner learns the value of the policy being carried out by the agent including the exploration steps."

The reason that Q-learning is off-policy is that it updates its Q-values using the Q-value of the next state s′ and the greedy action a′. In other words, it estimates the return (total discounted future reward) for state-action pairs assuming a greedy policy were followed despite the fact that it's not following a greedy policy.

The reason that SARSA is on-policy is that it updates its Q-values using the Q-value of the next state s′ and the current policy's action a′′. It estimates the return for state-action pairs assuming the current policy continues to be followed.